In [1]:
import pandas as pd 

df = pd.read_csv('artifacts/data.csv')
df = df[['GenHlth','MentHlth','PhysHlth']]
df.head()

,GenHlth,MentHlth,PhysHlth
0,5.0,18.0,15.0
1,3.0,0.0,0.0
2,5.0,30.0,30.0
3,2.0,0.0,0.0
4,2.0,3.0,0.0


In [7]:
df.max()

GenHlth      5.0
MentHlth    30.0
PhysHlth    30.0
dtype: float64

In [8]:
df.min()

GenHlth     1.0
MentHlth    0.0
PhysHlth    0.0
dtype: float64